## Data Sources
>1. The WeRateDogs Twitter archive. `twitter-archive-enhanced.csv` <br> 
 > 2. The tweet image predictions `image_prediction.tcv`<br> i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. <br>
 `url = https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv`
> 3. Info Extraccted from twitter API `tweet_api.csv` <br>
  Each tweet's retweet count and favorite ("like") count at minimum, and  any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.

## Data Gather
> 1. `twitter_archive_enhanced.csv` **imported from local**<br>
    The WeRateDogs Twitter archive contains basic tweet data for all 5000+ of their tweets, but not everything. One column the archive does contain though: each tweet's text, which I used to extract rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) to make this Twitter archive "enhanced." Of the 5000+ tweets, I have filtered for tweets with ratings only (there are 2356). <br>
     I extracted this data programmatically, but I didn't do a very good job. The ratings probably aren't all correct. Same goes for the dog names and probably dog stages (see below for more information on these) too. You'll need to assess and clean these columns if you want to use them for analysis and visualization. <br>
> 2. `image_prediction.tcv` **import from url via request.get**<br>
  One more cool thing: I ran every image in the WeRateDogs Twitter archive through a neural network that can classify breeds of dogs*. The results: a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images).
> 3. `twitter_api.csv` **query Twitter's API to get the json data and convert to csv file** <br>
  Back to the basic-ness of Twitter archives: retweet count and favorite count are two of the notable column omissions. Fortunately, this additional data can be gathered by anyone from Twitter's API. Well, "anyone" who has access to data for the 3000 most recent tweets, at least. 
  
  

## Data Assessing

#### Quality Issues

twitter_archive:<br>
1. retweets records present.
2. missing values on `in_reply_to_status_id`, `in_reply_to_user_id`,    `retweeted_status_id`, `retweeted_status_user_id`,  `retweeted_status_timestamp`, and `expanded_urls` 
3. `rating_numerator` and `rating_denominator` got extracted inaccurate, eg, date, address...
4. `name` got extracted inaccurated, eg 'a', 'the'..
5. `doggo`, `floofer`, `pupper`, `puppo` contain string 'none'
6. Erroneous datatypes for `timestamp`
7. `source` containes the whole tag of html, instead of the http address

imgae_prediction: <br>
1. `p1`, `p2` and `p3` contains '_' and mixed case.


#### Tidiness

twitter_archive:<br>
1. `doggo`, `floofer`, `pupper`, `puppo` these four columns represent the same thing, 'stage'

imgae_prediction: <br>
1. `p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog` these column represent the same thing, 'prediction'. 


## Data Assess and Data Cleaning
`twitter-archive-enhanced.csv`: After accessing and cleaning this dataset, I stored the data in `twitter-archive-enhanced-clean.csv`
> 1. check each row.<br> 
 What I checked is that each row represents one tweet_id and the other information associate with this tweet_id. 

> 2. Retweet records present
  - <font color = 'blue'>Data Cleaning: Drop the retweet rows  </font>

> 3. check each column for missing value<br>
 From the heatmap for missing values, we could tell that columns of `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, and `expanded_urls`. Among the total 2356 rows, the `in_reply_to_status_id` and `in_reply_to_user_id` share the same 2278 missing values since there are both t relatived to the act of in-reply.<br>
 Similar to retweeting, `retweeted_status_id`,`retweeted_status_user_id` and `retweeted_status_timestam` share the same pattern for 2175 missing values. `expanded_urls` has 59 missing values. Considering that we only want original ratings (no retweets) that have images. 
 - <font color = 'blue'>Data cleaning: dropping the columns mentioned above.</font>
 
> 4. Erroneous numeric variable <br>
     <mark> For detail, please see the document on function clean_rating in module twitter_archive_fix_rating </mark><br>
  `rating_numerator` and `rating_denominator` are numeric variables.
  The ratings are not all extracted correctly. So I tried to extract the ratings from the text from the pattern `'(\d+\/\d+)'` and these are 33 rows having multiple "ratings". Obvious, not all the multiple extractions are the correct ratings. <br>
   - Scenario 1:  some tweets contain the picture having more than one dogs, so there are multiple ratings for multiple dogs.<br>
 ex: These two pups just met and have instantly bonded. Spectacular scene. Mesmerizing af. 10/10 and 7/10 for blue dog.<br>
     - <font color = 'blue'>Data Cleaning for Scenario 1: I will drop these rows since it will against each row represents one rating. </font>
   - Scenario 2: the pattern `'(\d+\/\d+)'` matches date, location, etc, but not rating. <br>
  ex: After so many requests, this is Bretagne. She was the last surviving `9/11` search dog, and our second ever 14/10. RIP <br>
  ex: This is Darrel. He just robbed a `7/11` and is in a high speed police chase. Was just spotted by the helicopter 10/10 <br>
  ex: This is an Albanian 3 `1/2` legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10<br>
     - <font color = 'blue'>Data Cleaning for Scenario 2: dropping the wrong extraction, keep the correct dog rating. </font>
   - Scenario 3: After doing data cleaning for Scenario 1 and scenario 2, I maually check other erroneous `rating_numerator` and `rating_denominator` There are two twitters containing have wrong ratings. <br>
  For example, in the 342th tweet, the text is '@docmisterio account started on 11/15/15'. We could tell 11/15 got extracted to rating, but it actually a date.<br>
  Another example, in the 516th tweet, the text is 'Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link', which there is no actually rating but '24/7' got extracted. So I drop these texts without anything ratings, but something else got extracted like a date, 24/7, etc...<br>
  From the texts above, It seems like there are not give rating.
     - <font color = 'blue'>Data Cleaning:  Dropping the these two rows</font>
> 5. Columns `doggo`, `floofer`, `pupper`, `puppo`  <br>
  These four columns represents what kind dogstage (doggo, floofer, pupper, puppo). There are two values in doggo column, 'doggo' means the dogstage is doggo, 'None' means the dogstage is not doggo, Similar to doggo column, floofer, pupper and puppo follow the same pattern and then Dog stages columns (dogoo, puppu, pupper and floofer) should be merged into a single column as a tidiness issues.
   - <font color = 'blue'>Data Cleaning: mapping the 'none' to np.nan, mapping the corresponding stage to the stage name and merge to one column.</font><br>

> 6. Name misextraction<br>
   The column `name` is extracted from the twitter text. Some of the name are misextracted like 'a', 'None', 'the' ....
   - <font color = 'blue'>Data Cleaning : I find all the name starting lower case letter or None, and set them to missing value (np.nan) </font>
   
> 7. timestamp<br>
   The column `timestamp` is object, not timedate datatype.
   - <font color = 'blue'>Data Cleaning: convert the datetype to timedate.</font>
   
> 8. source <br>
  The column of source contains the whole tage of html. For example, the source in the first row is `<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>`. we could extract the address `http://twitter.com/download/iphone`.
  - <font color = 'blue'>Data Cleaning: extracting the http address only</font>
  


`image_prediction.csv` After assessing and cleaning dataset, I stored the data at `image_predition.csv`
> 1. Check if each row represents an unique tweet_id. 
     Yes, each row represents an unique tweet. 
> 2. No missing value
> 3. column p1, p2, p3 are mixed with '_' and case sensitive.<br>
  - <font color = 'blue'>Data Cleaning: I replace '_' (underscore) with ' ' (space), and make all characters lower case.</font>
> 4. Duplicated pictures <br>
   These are 66 duplicate rows for jpg_url, which means the first tweet is orginal tweet and later ones may be the retweet. meanwhile the duplicated picture will get the same result on the picture classification algorithm <br>
  - <font color = 'blue'> Data Cleaning: For the tweets contains same jpg_url, only keep the first one</font> <br>
> 5. Attribute Selection: <br>
  Since all of the pics are posted to 'WeRateDog' twitter, it seems like these are all containing dog/dogs. Therefore, the most likely prediction for each pic and it supposes to be a dog.
      - <font color = 'blue'>Data Cleaning: Selecting the most likely prediction if dog is True as breed, if all three predictions are not dog, then select p1, and then drop columns p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog</font>
  
  `tweet_api.csv` is no need to be cleaned.

## DataSet Merging
>Inner join 3 datasets to `twitter_archive_master.csv` for visualization<br>
